# FromageModel Inference Notebook 
## 0. Imports & Config

In [1]:
import torch
import torch.nn as nn
import yaml
import random
import sys
import os
import os.path as osp

/kuacc/users/oince22/.conda/envs/fromage/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from fromage.model import Fromage, FromageModel 
from fromage.experiment import Experiment
from fromage.data import MIMICDataset, COCODataset, cxr_image_transform, coco_image_transform
from fromage.utils import preprocess_report

In [3]:
ckpt_path = osp.join(osp.dirname(os.getcwd()), "logs/checkpoints/vl_eval_3/last.ckpt")
config_path = osp.join(osp.dirname(os.getcwd()), "config/train-vleval-3.yaml")
dataset_path = osp.join(osp.dirname(os.getcwd()), "data/MIMIC_JPG_train.tsv")
img_path = "/datasets/mimic/cxr-jpg/physionet.org/files/mimic-cxr-jpg/2.0.0/files"

## 1. Set device and config file

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

with open (config_path) as file:
    config = yaml.safe_load(file)

## 2. Load model and dataset

In [5]:
model = Experiment(config)
model = model.load_from_checkpoint(ckpt_path)
model = model.model.to(device)
model.device = device

In [6]:
transform = cxr_image_transform(resize=512, center_crop_size=480, train=False) 
dataset = MIMICDataset(dataset_path, img_path, transform)

## 3. Get random example report from dataset

In [30]:
ex_idx = random.randint(0, len(dataset) - 1)
ex_img, ex_report = dataset.__getitem__(ex_idx)
print(ex_report)

No such file: '/datasets/mimic/cxr-jpg/physionet.org/files/mimic-cxr-jpg/2.0.0/files/p18/p18085955/s55354095/a384aa24-4155c68b-4e23a82c-65dd3c3d-d54a67b7.jpg'
The heart is normal in size. The mediastinal and hilar contours are unremarkable. There is no pleural effusion or pneumothorax. The lungs are well-expanded and clear without focal consolidation concerning for pneumonia. A small nodular density at the right lung apex is noted, which may represent costochondral cartilage of the first rib or a nodule. The upper abdomen is unremarkable. There is no acute osseous abnormality. Nodular opacity at the right apex, which may represent costochondral cartilage versus an nodule. Shallow oblique films are recommended for further evaluation. The findings were discussed by Dr. with Dr on the telephone on at Nodular opacity at the right apex, which may represent costochondral cartilage versus an nodule. Shallow oblique films are recommended for further evaluation. The findings were discussed by D

## 4. Inference

In [31]:
with torch.inference_mode():
    model.eval()
    prompts = [ex_img, "Question: What does the previous image show? Answer: "] 
    print(model.generate_for_images_and_texts(prompts, top_p=0.9, temperature=0.5))

ipsipulmonary lesion. What is the next investigation? CT angiography. Micturating cystourethrogram. What is the next
